In [ ]:
# ライブラリインポート
from IPython.core.display import display
from IPython.core.display import Markdown as md
import datetime

In [ ]:
# 日付表示
display(md("creation date: {}".format(datetime.date.today())))

# AWS環境定義書
## 基本情報
- **環境** :: XXX用XXXAWS環境
- **アカウントID** :: DDDD-DDDD-DDDD
- **作成者** :: Kawahara Masahiro

In [ ]:
# ライブラリインポート
import pandas as pd
import boto3

In [ ]:
# 名前タグ取得用関数
def get_name_from_tags(tags):
    tags_filter = [t['Value'] for t in tags if t['Key'] == "Name"]
    if tags_filter:
        return tags_filter[0]
    else:
        return ""

## VPC

In [ ]:
ec2_client = boto3.client('ec2')

In [ ]:
# VPC 一覧取得
vpcs = ec2_client.describe_vpcs()['Vpcs']

In [ ]:
buffer = []
for vpc in vpcs:
    for association in vpc['CidrBlockAssociationSet']:
        buffer.append([
            get_name_from_tags(vpc['Tags']), 
            vpc['VpcId'], 
            association['CidrBlock'],
            vpc['InstanceTenancy']            
        ])
# Display
df_vpcs = pd.DataFrame(buffer, 
                       columns=["Name", "VpcId", "Cidr", "Tenancy"])
display(df_vpcs)

## RouteTables

In [ ]:
# RouteTable 一覧取得
rtbs = ec2_client.describe_route_tables()['RouteTables']

### RouteTable 一覧

In [ ]:
buffer = []
for rtb in rtbs:
    vpc = df_vpcs[df_vpcs['VpcId'] == rtb['VpcId']].iloc[0]
    # メインルートテーブルかどうかを調べる
    is_main = False
    for association in rtb['Associations']:
        if association['Main'] == True:
            is_main = True
    # append
    buffer.append([
        get_name_from_tags(rtb['Tags']),
        rtb['RouteTableId'],
        "{} | {}".format(vpc['VpcId'], vpc['Name']),
        is_main
    ])
# Display
df_rtbs = pd.DataFrame(buffer, 
                       columns=["Name", "RouteTableId", "Vpc", "Main"])
display(df_rtbs)

### Route一覧

In [ ]:
buffer = []
for rtb in rtbs:
    for route in rtb['Routes']:
        buffer.append([
            "{} | {}".format(rtb['RouteTableId'], get_name_from_tags(rtb['Tags'])),
            route['DestinationCidrBlock'],
            route['GatewayId']
        ])
# Display
df_rtbs_routes = pd.DataFrame(buffer, 
                               columns=["RouteTable", "Destination", "Gateway"])
display(df_rtbs_routes)

## Subnets

In [ ]:
# Subnet 一覧取得
subnets = ec2_client.describe_subnets()['Subnets']

In [ ]:
# サブネットとルートテーブル関連付け情報の取得
buffer = []
for rtb in rtbs:
    for association in rtb['Associations']:
        subnet_id = ""
        if association['Main'] == False:
            subnet_id = association['SubnetId']
        buffer.append([
            get_name_from_tags(rtb['Tags']),
            rtb['VpcId'],
            association['Main'],
            rtb['RouteTableId'],
            subnet_id
        ])
df_rtb_assoc = pd.DataFrame(buffer, 
                            columns=["Name", "VpcId", "Main", "RouteTableId", "SubnetId"])

In [ ]:
buffer = []
for subnet in subnets:
    vpc = df_vpcs[df_vpcs['VpcId'] == subnet['VpcId']].iloc[0]
    # RouteTable を調べる
    rtb_info = ""
    ## RouteTables の Association にない場合
    if df_rtb_assoc[df_rtb_assoc['SubnetId'] == subnet['SubnetId']].empty:
        ## VPCのデフォルトルートテーブルを取得
        default_rtb = df_rtb_assoc[
            (df_rtb_assoc['VpcId'] == subnet['VpcId'])
            & (df_rtb_assoc['Main'] == True)].iloc[0]
        rtb_info = "{} | {}".format(default_rtb['RouteTableId'], default_rtb['Name'])
    else:         
        ## RouteTables の Association にある場合
        rtb = df_rtb_assoc[df_rtb_assoc['SubnetId'] == subnet['SubnetId']].iloc[0]
        rtb_info = "{} | {}".format(rtb['RouteTableId'], rtb['Name'])
    # append
    buffer.append([
        get_name_from_tags(subnet['Tags']),
        subnet['SubnetId'],
        "{} ({}) | {}".format(vpc['VpcId'], vpc['Cidr'], vpc['Name']),
        subnet['CidrBlock'],
        subnet['AvailabilityZone'],
        rtb_info,
        subnet['MapPublicIpOnLaunch']
    ])
# Display
df_subnets = pd.DataFrame(buffer, 
                          columns=["Name", "SubnetId", "Vpc", "Cidr",
                                   "Az", "RouteTable", "PublicIpOnLaunch"])
display(df_subnets)

## EC2

In [ ]:
# EC2 インスタンス一覧取得
reservations = ec2_client.describe_instances()['Reservations']
instances = sum([resv['Instances'] for resv in reservations], [])

### EC2 インスタンス一覧

In [ ]:
buffer = []
for inst in instances:
    subnet = df_subnets[df_subnets['SubnetId'] == inst['SubnetId']].iloc[0]
    # append
    buffer.append([
        get_name_from_tags(inst['Tags']),
        inst['InstanceId'],
        inst['InstanceType'],
        "{} | {}".format(subnet['SubnetId'], subnet['Name']),
        inst['PublicIpAddress'] if 'PublicIpAddress' in inst else ''
    ])
# Display
df_instances = pd.DataFrame(buffer,
                            columns=["Name", "InstanceId", "InstanceType",
                                    "Subnet", "PublicIp"])
display(df_instances)

### インスタンス詳細

In [ ]:
for inst in instances:
    name = get_name_from_tags(inst['Tags'])
    # make list
    buffer = [[
        inst['InstanceId'],
        inst['InstanceType'],
        inst['PrivateDnsName'],
        inst['PrivateIpAddress'],
        inst['VpcId'],
        inst['SubnetId'],
        inst['SourceDestCheck'],
        inst['EbsOptimized'],
        inst['RootDeviceType'],
        inst['RootDeviceName'],
        inst['PublicDnsName'],
        inst['PublicIpAddress'] if 'PublicIpAddress' in inst else '',
        inst['Placement']['AvailabilityZone'],
        ", ".join([ sg['GroupName'] for sg in inst['SecurityGroups']]),
        inst['IamInstanceProfile']['Arn'].split('/')[-1],
        inst['ImageId']
    ]]
    df_instance = pd.DataFrame(buffer, 
                               index = ["value"],
                               columns=["InstanceId", "InstanceType", "PrivateDnsName",
                                       "PrivateIpAddress", "VpcId", "SubnetId",
                                       "SourceDestCheck", "EbsOptimized", "RootDeviceType",
                                       "RootDeviceName", "PublicDnsName", "PublicIpAddress",
                                       "AvailabilityZone", "SecurityGroups", "IamRole",
                                       "ImageId"])
    # display
    display(md("#### " + name))
    display(df_instance.T)